

TensorFlow Quantum example, adapted from
[*TensorFlow Quantum: A Software Framework for Quantum Machine Learning*](http://arxiv.org/abs/2003.02989)

Loading of libraries and initialization

In [1]:
import random

import cirq
from cirq.contrib.svg import SVGCircuit
import matplotlib.pyplot as plt
import numpy as np
import sympy
import tensorflow as tf
import tensorflow_quantum as tfq

if not tf.config.list_physical_devices('GPU'):
    print("Warning: GPU was not found, so simulations can be very slow")

TensorFlow Quantum example

In [6]:
def generate_dataset(
        qubit, theta_a, theta_b, num_samples):
    q_data = []
    labels = []
    blob_size = abs(theta_a - theta_b) / 5
    for _ in range(num_samples):
        coin = random.random()
        spread_x, spread_y = np.random.uniform(-blob_size, blob_size, 2)
        if coin < 0.5:
            label = [1, 0]
            angle = theta_a + spread_y
        else:
            label = [0, 1]
            angle = theta_b + spread_y
        labels.append(label)
        q_data.append(cirq.Circuit(
            cirq.Ry(rads=-angle)(qubit),
            cirq.Rx(rads=-spread_x)(qubit)
        ))
    return tfq.convert_to_tensor(q_data), np.array(labels)

# Dataset generation
qubit = cirq.GridQubit(0, 0)
theta_a = 1
theta_b = 4
num_samples = 200
q_data, labels = generate_dataset(qubit, theta_a, theta_b, num_samples)

# Quantum parametric model
theta = sympy.Symbol("theta")
q_model = cirq.Circuit(cirq.Ry(rads=theta)(qubit))
q_data_input = tf.keras.Input(shape=(), dtype=tf.dtypes.string)
expectation = tfq.layers.PQC(q_model, cirq.Z(qubit))
expectation_output = expectation(q_data_input)

classifier = tf.keras.layers.Dense(2, activation=tf.keras.activations.softmax)
classifier_output = classifier(expectation_output)
model = tf.keras.Model(inputs=q_data_input, outputs=classifier_output)

optimizer = tf.keras.optimizers.Adam(learning_rate=0.1)

loss = tf.keras.losses.CategoricalCrossentropy()
model.compile(optimizer=optimizer, loss=loss)
history = model.fit(x=q_data, y=labels, epochs=50)

test_data, _ = generate_dataset(qubit, theta_a, theta_b, 1)
p = model.predict(test_data)[0]
print(f"prob(a)={p[0]:.4f}, prob(b)={p[1]:.4f}")

Epoch 1/50
7/7 [==============================] - 1s 5ms/step - loss: 0.7619
Epoch 2/50
7/7 [==============================] - 0s 5ms/step - loss: 0.3277
Epoch 3/50
7/7 [==============================] - 0s 5ms/step - loss: 0.1140
Epoch 4/50
7/7 [==============================] - 0s 4ms/step - loss: 0.0576
Epoch 5/50
7/7 [==============================] - 0s 4ms/step - loss: 0.0306
Epoch 6/50
7/7 [==============================] - 0s 5ms/step - loss: 0.0201
Epoch 7/50
7/7 [==============================] - 0s 4ms/step - loss: 0.0157
Epoch 8/50
7/7 [==============================] - 0s 5ms/step 